In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime

# Load Cifar10 dataset

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [19]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 50
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [8]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [9]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [10]:
val_split = int((1-VAL_SPLIT)*len(x_train))
x_train , x_val = x_train[:val_split], x_train[val_split:]
y_train, y_val = y_train[:val_split], y_train[val_split:]

# Define data augmentations

In [11]:
datagen = ImageDataGenerator(width_shift_range=.2, 
                             height_shift_range=.2,
                             zoom_range=.2,
                             horizontal_flip=True, 
                             vertical_flip=True)

# Build model

In [12]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [13]:
base_model = VGG16(include_top=False, input_tensor=input_tensor)

In [15]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [16]:
vgg16 = Model(inputs=base_model.input, outputs=predictions)

In [17]:
vgg16.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [18]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [20]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [21]:
vgg16.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) / BATCH_SIZE,
                    validation_data=ImageDataGenerator().flow(x_val, y_val, batch_size=BATCH_SIZE),
                    validation_steps=len(x_val) / BATCH_SIZE,
                    epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/50
79/78 [==============================] - 43s 549ms/step - loss: 2.2816 - acc: 0.1475 - val_loss: 2.1963 - val_acc: 0.1329
Epoch 2/50
79/78 [==============================] - 33s 421ms/step - loss: 1.9950 - acc: 0.2027 - val_loss: 1.8518 - val_acc: 0.2551
Epoch 3/50
79/78 [==============================] - 35s 442ms/step - loss: 1.8087 - acc: 0.2854 - val_loss: 1.6090 - val_acc: 0.3670
Epoch 4/50
79/78 [==============================] - 34s 431ms/step - loss: 1.5381 - acc: 0.4038 - val_loss: 1.4121 - val_acc: 0.4857
Epoch 5/50
79/78 [==============================] - 34s 433ms/step - loss: 1.3058 - acc: 0.5157 - val_loss: 1.1018 - val_acc: 0.6105
Epoch 6/50
79/78 [==============================] - 35s 448ms/step - loss: 1.1017 - acc: 0.6032 - val_loss: 0.9820 - val_acc: 0.6633
Epoch 7/50
79/78 [==============================] - 34s 432ms/step - loss: 0.9693 - acc: 0.6548 - val_loss: 0.8446 - val_acc: 0.7078
Epoch 8/50
79/78 [==============================] - 35s 446ms/step - 

In [22]:
vgg16.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 6s 577us/step


[0.4318272342205048, 0.8604]